## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-04-22-55-10 +0000,missionlocal,They want to be the next California governor. ...,https://missionlocal.org/2026/02/california-go...
1,2026-02-04-22-54-45 +0000,cbc,Federal bill wants to make it easier to share ...,https://www.cbc.ca/news/health/health-system-m...
2,2026-02-04-22-54-02 +0000,nyt,"Nike, Accused of Bias Against White Workers, I...",https://www.nytimes.com/2026/02/04/business/ee...
3,2026-02-04-22-51-26 +0000,nypost,3 sisters jump to their deaths from balcony af...,https://nypost.com/2026/02/04/world-news/3-sis...
4,2026-02-04-22-49-00 +0000,wsj,"U.S., Iran Agree to Discuss Nuclear Issues Fri...",https://www.wsj.com/world/middle-east/iran-pla...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2456/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
91,trump,40
270,epstein,25
119,new,24
861,house,14
184,deal,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
362,2026-02-04-01-07-00 +0000,wsj,Fed governor Stephen Miran said he resigned fr...,https://www.wsj.com/economy/central-banking/st...,117
375,2026-02-04-00-00-00 +0000,wsj,Stephen Miller called Alex Pretti a “domestic ...,https://www.wsj.com/politics/policy/ice-stephe...,116
130,2026-02-04-19-57-31 +0000,nypost,President Trump called Savannah Guthrie to off...,https://nypost.com/2026/02/04/us-news/presiden...,97
383,2026-02-03-23-21-07 +0000,bbc,Trump hails White House talks with 'terrific' ...,https://www.bbc.com/news/articles/c4g01vlwqp5o...,86
202,2026-02-04-16-39-42 +0000,nypost,New Epstein emails about top Goldman lawyer Ka...,https://nypost.com/2026/02/04/business/epstein...,80


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
362,117,2026-02-04-01-07-00 +0000,wsj,Fed governor Stephen Miran said he resigned fr...,https://www.wsj.com/economy/central-banking/st...
375,84,2026-02-04-00-00-00 +0000,wsj,Stephen Miller called Alex Pretti a “domestic ...,https://www.wsj.com/politics/policy/ice-stephe...
202,74,2026-02-04-16-39-42 +0000,nypost,New Epstein emails about top Goldman lawyer Ka...,https://nypost.com/2026/02/04/business/epstein...
369,53,2026-02-04-00-31-15 +0000,nypost,Netflix co-CEO grilled by senators as Warner B...,https://nypost.com/2026/02/03/business/netflix...
169,47,2026-02-04-18-13-46 +0000,cbc,Washington Post staff laid off while in Milan ...,https://www.cbc.ca/sports/olympics/winter/mila...
150,45,2026-02-04-19-10-44 +0000,latimes,"Supreme Court, with no dissents, rejects GOP c...",https://www.latimes.com/politics/story/2026-02...
231,42,2026-02-04-15-00-14 +0000,nyt,Minneapolis Police Chief Fears Long-Term Damag...,https://www.nytimes.com/2026/02/04/us/brian-oh...
234,42,2026-02-04-14-52-00 +0000,wsj,"America’s private sector added 22,000 jobs las...",https://www.wsj.com/economy/jobs/adp-numbers-s...
240,42,2026-02-04-14-13-10 +0000,startribune,Live: Border official Tom Homan announces hund...,https://www.startribune.com/ice-raids-minnesot...
85,39,2026-02-04-21-09-17 +0000,nyt,Former Prince Andrew Moves Out of Royal Home A...,https://www.nytimes.com/2026/02/04/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
